reference : https://github.com/boostcampaitech2/semantic-segmentation-level2-cv-04

In [13]:
import os
import os.path as osp
import shutil
import time
import warnings

import mmcv
import mmcv_custom   # noqa: F401,F403
import mmseg_custom   # noqa: F401,F403
import torch
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmcv.runner import (get_dist_info, init_dist, load_checkpoint,
                         wrap_fp16_model)
from mmcv.utils import DictAction
from mmseg.apis import multi_gpu_test, single_gpu_test
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor

import pandas as pd
import numpy as np
import json

In [6]:
# config file_dir, work_dir 수정 후 사용

# config file 들고오기
cfg = Config.fromfile('/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-19/ViT-Adapter/segmentation/configs/aistage/mask2former_beit_adapter_base_512_20k_aistage_ss.py') # load config file
root='/opt/ml/input/mmseg/test/' # test img
epoch = 'best_mIoU_iter_20000' # pth filename

# dataset config 수정
cfg.work_dir = './work_dirs/mask2former_beit_adapter_base_512_20k_aistage_ss/' # set work_dir
cfg.data.test.img_dir = root
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True
cfg.data.samples_per_gpu = 1
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

In [7]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=cfg.data.samples_per_gpu,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

2023-01-03 01:56:51,216 - mmseg - INFO - Loaded 819 images


In [8]:
model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2023-01-03 01:56:53,157 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc_norm.weight, fc_norm.bias, head.weight, head.bias

missing keys in source state_dict: blocks.0.attn.relative_position_index, blocks.1.attn.relative_position_index, blocks.2.attn.relative_position_index, blocks.3.attn.relative_position_index, blocks.4.attn.relative_position_index, blocks.5.attn.relative_position_index, blocks.6.attn.relative_position_index, blocks.7.attn.relative_position_index, blocks.8.attn.relative_position_index, blocks.9.attn.relative_position_index, blocks.10.attn.relative_position_index, blocks.11.attn.relative_position_index



Position interpolate for blocks.0.attn.relative_position_bias_table from 27x27 to 63x63
x = [-31.000181806480445, -26.433792368127026, -22.410250408812363, -18.86502280242512, -15.741248051675145, -12.988824293086934, -10.563605718981357, -8.426694527950382, -6.5438170474999815, -4.884774022564157, -3.422956253130848, -2.134917812347412, -1, 0, 1, 2.134917812347412, 3.422956253130848, 4.884774022564157, 6.5438170474999815, 8.426694527950382, 10.563605718981357, 12.988824293086934, 15.741248051675145, 18.86502280242512, 22.410250408812363, 26.433792368127026, 31.000181806480445]
dx = [-31. -30. -29. -28. -27. -26. -25. -24. -23. -22. -21. -20. -19. -18.
 -17. -16. -15. -14. -13. -12. -11. -10.  -9.  -8.  -7.  -6.  -5.  -4.
  -3.  -2.  -1.   0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.
  11.  12.  13.  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.
  25.  26.  27.  28.  29.  30.  31.]
Position interpolate for blocks.1.attn.relative_position_bias_table from 27x27 to 63x63
x

/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-19/ViT-Adapter/segmentation/mmseg_custom/models/losses/cross_entropy_loss.py:230: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from local path: ./work_dirs/mask2former_beit_adapter_base_512_20k_aistage_ss/best_mIoU_iter_20000.pth


In [9]:
output = single_gpu_test(model, data_loader)

[                                 ] 1/819, 1.0 task/s, elapsed: 1s, ETA:   855s

/opt/conda/envs/vita/lib/python3.8/site-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 819/819, 7.6 task/s, elapsed: 107s, ETA:     0s

In [12]:
# sample_submisson.csv 열기
submission = pd.read_csv('../../submission/sample_submission.csv', index_col=None)
json_dir = os.path.join("/opt/ml/input/data/test.json")
with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)

input_size = 512
output_size = 256
bin_size = input_size // output_size
		
# PredictionString 대입
for image_id, predict in enumerate(output):
    image_id = datas["images"][image_id]
    file_name = image_id["file_name"]
    
    temp_mask = []
    predict = predict.reshape(1, 512, 512)
    # resize predict to 256, 256
    # reference : https://stackoverflow.com/questions/48121916/numpy-resize-rescale-image
    mask = predict.reshape((1, output_size, bin_size, output_size, bin_size)).max(4).max(2) 
    temp_mask.append(mask)
    oms = np.array(temp_mask)
    oms = oms.reshape([oms.shape[0], output_size*output_size]).astype(int)

    string = oms.flatten()

    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=False)

In [15]:
cfg.work_dir

'./work_dirs/mask2former_beit_adapter_base_512_20k_aistage_ss/'